In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [2]:
# Step 1: Open LinkedIn login page
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.linkedin.com/login")


In [4]:
# Step 3: Start scraping jobs
with open('linkedin_jobs_with_insights_loop4.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow([
        "Job Title", "Company Name", "Job Location", "Easy Apply", "Job Link",
        "Insight 1", "Insight 2", "Insight 3",
        "Responsibilities", "Qualifications", "Benefits", "Job Summary"
    ])

    job_links = []

    for start in range(0, 25, 25):
        url = f"https://www.linkedin.com/jobs/search/?keywords=data%20analyst&location=Egypt&start={start}"
        driver.get(url)
        time.sleep(5)

        for _ in range(5):
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
            time.sleep(2)

        job_blocks = driver.find_elements(By.CLASS_NAME, "job-card-container")
        print(f"📄 Page {start} contains {len(job_blocks)} jobs")

        for block in job_blocks:
            try:
                link = block.find_element(By.CSS_SELECTOR, 'a.job-card-container__link').get_attribute("href")
                job_links.append(link)
            except:
                continue

    # Move this loop INSIDE the 'with open' block
    for job_url in job_links:
        driver.get(job_url)
        time.sleep(3)

        try:
            job_title = driver.find_element(By.CSS_SELECTOR, 'h1').text.strip()
        except:
            job_title = "N/A"

        try:
            company_name = driver.find_element(By.CSS_SELECTOR, 'a.topcard__org-name-link').text.strip()
        except:
            try:
                company_name = driver.find_element(By.CSS_SELECTOR, 'span.topcard__flavor').text.strip()
            except:
                company_name = "N/A"

        try:
            job_location = driver.find_element(By.CSS_SELECTOR, 'span.topcard__flavor.topcard__flavor--bullet').text.strip()
        except:
            job_location = "N/A"

        easy_apply = "Easy Apply" if "Easy Apply" in driver.page_source else "Not Available"

        try:
            insights_block = driver.find_element(By.XPATH, "//li[contains(@class, 'job-insight')]")
            insights_texts = insights_block.find_elements(By.XPATH, ".//span[not(contains(@class, 'secondary')) and @dir='ltr'] | .//span[contains(@class, 'secondary')]")
            insight_values = [insight.text.strip() for insight in insights_texts if insight.text.strip()]
            while len(insight_values) < 3:
                insight_values.append("N/A")
        except:
            insight_values = ["N/A", "N/A", "N/A"]

        try:
            jd_container = driver.find_element(By.ID, 'job-details')
            jd_html = jd_container.get_attribute("innerHTML")
            from bs4 import BeautifulSoup
            soup = BeautifulSoup(jd_html, 'html.parser')

            def extract_section(header_keywords):
                for strong in soup.find_all(['strong', 'h3', 'h4']):
                    if any(kw in strong.text.lower() for kw in header_keywords):
                        ul = strong.find_parent().find_next('ul')
                        if ul:
                            return "\n".join(li.text.strip() for li in ul.find_all('li'))
                        p = strong.find_parent().find_next('p')
                        if p:
                            return p.text.strip()
                return "N/A"

            job_summary = soup.text.strip()[:400]
            responsibilities = extract_section(['responsibilities', 'tasks', 'role'])
            qualifications = extract_section(['qualification', 'requirements', 'skills'])
            benefits = extract_section(['benefit', 'perks'])

        except Exception as e:
            job_summary = responsibilities = qualifications = benefits = "N/A"

        writer.writerow([
            job_title, company_name, job_location, easy_apply, job_url,
            insight_values[0], insight_values[1], insight_values[2],
            responsibilities, qualifications, benefits, job_summary
        ])

driver.quit()


📄 Page 0 contains 25 jobs
